In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Assuming your reshaped dataset is loaded into a DataFrame named 'melted_df'
# Replace 'path/to/your/reshaped_dataset.csv' with the actual path to your reshaped dataset
melted_df = pd.read_csv('/content/Transformed data.csv')
melted_df['Date'] = pd.to_datetime(melted_df['Date'])
# Choose a specific currency for modeling (e.g., 'USD')
currency_code = 'Australian dollar   (AUD)                     '
currency_df = melted_df[melted_df['Currency'] == currency_code]['ExchangeRate']

# Check stationarity
def test_stationarity(timeseries):
    # Perform Dickey-Fuller test:
    result = adfuller(timeseries, autolag='AIC')
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    print('Critical Values:', result[4])
    print("Is the time series stationary? ", result[1] < 0.05)

# Function to plot ACF and PACF
def plot_acf_pacf(timeseries, lags=30):
    fig, ax = plt.subplots(1, 2, figsize=(12, 4))
    plot_acf(timeseries, lags=lags, ax=ax[0])
    plot_pacf(timeseries, lags=lags, ax=ax[1])
    plt.show()

# Check stationarity
test_stationarity(currency_df)

# If not stationary, apply differencing
currency_df_diff = currency_df.diff().dropna()
test_stationarity(currency_df_diff)

# Plot ACF and PACF for differenced series
plot_acf_pacf(currency_df_diff)

# Choose p, d, q parameters based on ACF and PACF plots
p = 1  # lag value where PACF chart crosses the upper confidence interval for the first time
d = 1  # differencing order (1 for first difference)
q = 1  # lag value where ACF chart crosses the upper confidence interval for the first time

# Fit SARIMA model
model = SARIMAX(currency_df, order=(p, d, q), seasonal_order=(1, 0, 1, 12))
results = model.fit(disp=False)

# Make predictions
start_date = pd.Timestamp(currency_df.index.max()) + pd.DateOffset(days=1)
end_date = start_date + pd.DateOffset(months=3)  # Adjust prediction horizon as needed
forecast = results.get_forecast(steps=(end_date - start_date).days)
predicted_mean = forecast.predicted_mean
print(predicted_mean)